In [ ]:
import arkouda as ak
import arachne as ar

In [ ]:
ak.connect("n117", 5555)
ak.client.maxTransferBytes *= 2

In [3]:
connectome_edges = ak.read("/scratch/users/oaa9/experimentation/data/connectome/h01/edges*")
connectome_edges = ak.DataFrame(connectome_edges)

In [4]:
graph = ar.PropGraph()
graph.load_edge_attributes(connectome_edges, source_column="src_neuron_id", destination_column="dst_neuron_id")

In [5]:
num_nodes = len(graph)
num_edges = graph.size()

In [6]:
P_Alpha = 0.6
P_Beta = 0.4
node_lbl_probs = {
    "lbls2": [P_Alpha, P_Beta],  # Probabilities for integers 10 and 11
    "lbls3": [P_Alpha, P_Beta]   # Probabilities for True and False
}
edge_rel_probs = {
    "rels1": [P_Alpha, P_Beta],  # Probabilities for integers 5 and 6
    "rels2": [P_Alpha, P_Beta]   # Probabilities for True and False
}

In [7]:
# Randomly generate node attributes.
node_lbls2 = ak.where(ak.randint(0, 100, num_nodes) < node_lbl_probs["lbls2"][0] * 100, 10, 11)
node_lbls3 = ak.randint(0, 100, num_nodes) < node_lbl_probs["lbls3"][0] * 100

# Randomly generate edge attributes
edge_rels1 = ak.where(ak.randint(0, 100, num_edges) < edge_rel_probs["rels1"][0] * 100, 5, 6)
edge_rels2 = ak.randint(0, 100, num_edges) < edge_rel_probs["rels2"][0] * 100

In [ ]:
# Create edge dataframe.
edge_df = ak.DataFrame({
    "src": graph.edge_attributes["src_neuron_id"],
    "dst": graph.edge_attributes["dst_neuron_id"],
    "rels1": edge_rels1,
    "rels2": edge_rels2
})

# Create node dataframe.
node_df = ak.DataFrame({
    "nodes": graph.nodes(),
    "lbls2": node_lbls2,
    "lbls3": node_lbls3
})

In [ ]:
graph = ar.PropGraph()
graph.load_edge_attributes(edge_df, source_column="src", destination_column="dst")
graph.load_node_attributes(node_df, node_column="nodes")

In [12]:
def save_graph_to_csv(graph, file_name):
    """
    Save a graph in CSV format with directed edges and optional labels.
    :param graph: Arachne property graph.
    :param file_name: Output CSV file name.
    """
    # Extract internal edges and attributes
    internal_src, internal_dst = graph._internal_edges()
    src = internal_src.to_list()
    dst = internal_dst.to_list()

    # Extract edge attributes
    graph_edge_attributes = graph.get_edge_attributes()
    edge_df = graph_edge_attributes.to_pandas()
    edge_rels1 = edge_df['rels1'] if 'rels1' in edge_df.columns else None
    edge_rels2 = edge_df['rels2'] if 'rels2' in edge_df.columns else None
    
    with open(file_name, "w") as f:
        print("Writing edge data")
        for i in range(len(src)):
            if edge_rels1 is not None and edge_rels2 is not None:
                f.write(f"{src[i]}>{dst[i]},{edge_rels1[i]},{edge_rels2[i]}\n")
            else:
                f.write(f"{src[i]}>{dst[i]}\n")

        print("Writing node data...")
        graph_node_attributes = graph.get_node_attributes()
        if graph_node_attributes.size > 0:
            node_df = graph_node_attributes.to_pandas()
            for index, row in node_df.iterrows():
                f.write(f"{index},,{row['lbls2']},{row['lbls3']}\n")
        else:
            # Use default labels if no node attributes are available
            unique_nodes = sorted(set(src).union(dst))
            for node in unique_nodes:
                f.write(f"{node},,\n")

    print(f"Graph saved to {file_name}")

In [ ]:
save_graph_to_csv(graph, "H01.csv")